In [ ]:
from langchain.chat_models import ChatOpenAI

# import message constructor
from langchain.schema import HumanMessage, AIMessage, SystemMessage

chat = ChatOpenAI(temperature=0.1)

In [13]:
messages = [
    SystemMessage(content='You are a geography expert. And you only reply in Korean.'),
    AIMessage(content='Ciao, mi chiamo Pablo!'),
    HumanMessage(content='What is the distance between Mexico and Thailand. Also, what is your name?')
]

chat.predict_messages(messages)

AIMessage(content='안녕하세요! 멕시코와 태국 사이의 거리는 대략 16,000km입니다. 저는 지리 전문가로, 이름은 고객님의 질문에 대한 답변을 제공하는 AI 어시스턴트입니다. 어떤 도움이 필요하신가요?')

# Prompt Templates

In [14]:
from langchain.prompts import PromptTemplate, ChatPromptTemplate

In [16]:
template = PromptTemplate.from_template('What is the distance between {country_a} and {country_b}')
prompt = template.format(country_a='Mexico', country_b='Thailand')
chat.predict(prompt)

'The distance between Mexico and Thailand is approximately 9,500 miles (15,300 kilometers) when measured in a straight line.'

In [18]:
template = ChatPromptTemplate.from_messages([
    ('system','You are a geography expert. And you only reply in {language}.'),
    ('ai','Ciao, mi chiamo {name}!'),
    ('human', 'What is the distance between {country_a} and {country_b}. Also, what is your name?')
])

prompt = template.format_messages(
    language='Korean',
    name='Woojin',
    country_a='Korea',
    country_b='Japan'
)

chat.predict_messages(prompt)

AIMessage(content='한국과 일본 사이의 거리는 약 900km입니다. 제 이름은 Woojin입니다.')

# output parser and LCEL

In [ ]:
from langchain.schema import BaseOutputParser

class CommaOutputParser(BaseOutputParser):
    def parse(self, text):
        items = text.strip().split(',')
        return list(map(str.strip,items))

# p = CommaOutputParser()
# p.parse('Hello, how, are, you')

['Hello', 'how', 'are', 'you']

In [26]:
template = ChatPromptTemplate.from_messages([
    ('system','You are a list generating machine. Everything you are asked will be answered will be answered with a comma separted list of max {max_items}. Do Not replay with anything else.'),
    ('human','{question}')
])

prompt = template.format_messages(max_items=10, question='what are the colors?')

result = chat.predict_messages(prompt)
print(result)

content='red, blue, green, yellow, orange, purple, pink, black, white, brown'


In [27]:
p = CommaOutputParser()
p.parse(result.content)

['red',
 'blue',
 'green',
 'yellow',
 'orange',
 'purple',
 'pink',
 'black',
 'white',
 'brown']

# create chain

In [29]:
chain = template | chat | CommaOutputParser()
chain.invoke({'max_items':5,
                'question':'what are the pokemons?'})

['Pikachu', 'Charizard', 'Bulbasaur', 'Squirtle', 'Jigglypuff']

# LangChain

In [ ]:
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(temperature=0.1,
                streaming=True, # allow to look at the process
                callbacks=[StreamingStdOutCallbackHandler()])

In [34]:
chef_prompt = ChatPromptTemplate.from_messages([
    ('system','You are a world-class international chef. You create easy to follow recipes for any type of cuisine with easy to find ingredients.'),
    ('human','I want to cook {cuisine} food.')
])

chef_chain = chef_prompt | chat

In [35]:
veg_chef_prompt = ChatPromptTemplate.from_messages([
    ('system','You are a vegetarian chef specialized on making traditional recipies vegetarian. Yoy find alternative ingredients and explain their preparation.\
         You don\'t radically modify the recipie. If there is no alternative for a food just say you don\'t know how to replace it.'),
    ('human','{recipe}')
])

veg_chain = veg_chef_prompt | chat

In [ ]:
final_chain = {'recipe': chef_chain} | veg_chain   # runnable map : run previous one first and then use that output as input of the next step
response = final_chain.invoke({'cuisine':'indian'})

Great choice! Indian cuisine is known for its bold flavors and aromatic spices. Let's start with a classic and popular dish - Chicken Tikka Masala. Here's a simple recipe for you to try at home:

Ingredients:
- 1 lb boneless, skinless chicken breasts, cut into bite-sized pieces
- 1 cup plain yogurt
- 2 tablespoons lemon juice
- 2 tablespoons vegetable oil
- 1 onion, finely chopped
- 3 cloves garlic, minced
- 1 tablespoon ginger, minced
- 1 can (14 oz) tomato sauce
- 1 tablespoon garam masala
- 1 teaspoon ground cumin
- 1 teaspoon ground coriander
- 1 teaspoon paprika
- 1/2 teaspoon turmeric
- 1/2 teaspoon cayenne pepper (adjust to taste)
- Salt and pepper to taste
- Fresh cilantro, chopped (for garnish)
- Cooked rice or naan bread (to serve)

Instructions:
1. In a bowl, combine the yogurt, lemon juice, 1 tablespoon of vegetable oil, half of the minced garlic, half of the minced ginger, and a pinch of salt. Add the chicken pieces and mix well to coat. Cover and marinate in the refrigera

In [39]:
print(response.content)

To make this Chicken Tikka Masala recipe vegetarian, we can replace the chicken with a suitable alternative. One popular and delicious option is to use paneer, which is a type of Indian cottage cheese that holds its shape well when cooked. Here's how you can prepare the paneer as a substitute for chicken in this recipe:

Alternative Ingredient:
- 1 lb paneer, cut into bite-sized cubes

Instructions for Paneer Preparation:
1. Cut the paneer into bite-sized cubes and set aside.
2. In a bowl, combine the plain yogurt, lemon juice, 1 tablespoon of vegetable oil, half of the minced garlic, half of the minced ginger, and a pinch of salt. Add the paneer cubes and gently toss to coat. Cover and marinate in the refrigerator for at least 1 hour, or overnight for best results.
3. Preheat the oven to 400°F (200°C). Thread the marinated paneer cubes onto skewers and place them on a baking sheet lined with aluminum foil. Bake for about 15-20 minutes, or until the paneer is slightly golden and heated

# FewShotPromptTemplate

In [41]:
from langchain.prompts.few_shot import FewShotPromptTemplate

In [ ]:
# t = PromptTemplate(
#     template="What is the capital of {country}",
#     input_variables=["country"]
#     )

t = PromptTemplate.from_template("What is the capital of {country}")
t.format(country="France")

'What is the capital of France'

In [51]:
examples = [
    {
        "question":"what do you know about France?",
        "answer":"""
        Here is what I know:
        Capital: Paris
        Language: Franch
        Food: Wine and Cheese
        Currency: Euro
        """
    },
    {
        "question":"What do you know about Italy?",
        "answer":"""
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """
    },
    {
        "question":"What do you know about Greece?",
        "answer":"""
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """
    }
]

In [ ]:
# answer without few shot examples
response = chat.predict("what do you know about France?")
print(response)

France is a country located in Western Europe. It is known for its rich history, culture, and cuisine. The capital city is Paris, which is famous for landmarks such as the Eiffel Tower, Louvre Museum, and Notre-Dame Cathedral.

France is the largest country in the European Union by land area and the third-largest in Europe overall. It has a population of around 67 million people. The official language is French.

France is a popular tourist destination, attracting millions of visitors each year to its beautiful cities, countryside, and coastline. It is also known for its fashion, art, and architecture.

The country has a long history of influential art movements, including Impressionism, Cubism, and Surrealism. French cuisine is also highly regarded around the world, with dishes such as croissants, baguettes, and coq au vin being popular.

France is a founding member of the United Nations, NATO, and the European Union. It is also a nuclear-armed state and has a permanent seat on the UN

In [52]:
example_template = """
Human: {question}
AI: {answer}
"""

example_prompt = PromptTemplate.from_template(example_template)

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
    suffix="Human: What do you know about {country}?",
    input_variables=["country"]
)

print(prompt.format(country="Germany"))


Human: what do you know about France?
AI: 
        Here is what I know:
        Capital: Paris
        Language: Franch
        Food: Wine and Cheese
        Currency: Euro
        



Human: What do you know about Italy?
AI: 
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        



Human: What do you know about Greece?
AI: 
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        


Human: What do you know about Germany?


In [54]:
chain = prompt | chat
chain.invoke({'country':'Korea'})

AI: 
        I know this:
        Capital: Seoul
        Language: Korean
        Food: Kimchi and Bibimbap
        Currency: South Korean Won

AIMessageChunk(content='AI: \n        I know this:\n        Capital: Seoul\n        Language: Korean\n        Food: Kimchi and Bibimbap\n        Currency: South Korean Won')

In [56]:
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.prompts import ChatPromptTemplate

In [ ]:
examples = [
    {
        "country": "France",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "country": "Italy",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "country": "Greece",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]


example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "What do you know about {country}?"),
        ("ai", "{answer}"),
    ]
)

example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a geography expert, you give short answers."),
        example_prompt,
        ("human", "What do you know about {country}?"),
    ]
)

chain = final_prompt | chat

chain.invoke({"country": "Germany"})


        I know this:
        Capital: Berlin
        Language: German
        Food: Bratwurst and Sauerkraut
        Currency: Euro
        

AIMessageChunk(content='\n        I know this:\n        Capital: Berlin\n        Language: German\n        Food: Bratwurst and Sauerkraut\n        Currency: Euro\n        ')

# Length based Example Selector

how to select examples based on the length (set the limit of exmple length)

In [ ]:
from langchain.prompts.example_selector import LengthBasedExampleSelector

examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]


example_prompt = PromptTemplate.from_template("Human:{question}\nAI:{answer}")

example_selector = LengthBasedExampleSelector(
    examples=examples,
    example_prompt=example_prompt,
    max_length=180,
)

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    example_selector=example_selector,
    suffix="Human: What do you know about {country}?",
    input_variables=['country']
)

print(prompt.format(country="Brazil"))

Human:What do you know about France?
AI:
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        

Human:What do you know about Italy?
AI:
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        

Human: What do you know about Brazil?


- create our own random example selector

In [ ]:
from langchain.prompts.example_selector.base import BaseExampleSelector

class RandomExampleSelector(BaseExampleSelector):

    def __init__(self, examples):
        self.examples = examples

    def add_example(self, example):
        self.examples.append(example)

    def select_examples(self, input_variables):
        from random import choice
        return [choice(self.examples)]

example_selector = RandomExampleSelector(
    examples=examples,
)

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    example_selector=example_selector,
    suffix="Human: What do you know about {country}?",
    input_variables=['country']
)

print(prompt.format(country="Brazil"))

Human:What do you know about Italy?
AI:
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        

Human: What do you know about Brazil?


# Serialization and Composition

In [84]:
from langchain.prompts import load_prompt

# prompt = load_prompt('./prompt.json')
prompt = load_prompt('./prompt.yaml')
prompt.format(country="Germany")

'What is the capital of Germany'

In [86]:
from langchain.prompts.pipeline import PipelinePromptTemplate

intro = PromptTemplate.from_template(
    """
    You are a role playing assistant.
    And you are impersonating a {character}
"""
)

example = PromptTemplate.from_template(
    """
    This is an example of how you talk:

    Human: {example_question}
    You: {example_answer}
"""
)

start = PromptTemplate.from_template(
    """
    Start now!

    Human: {question}
    You:
"""
)

final = PromptTemplate.from_template(
    """
    {intro}
                                     
    {example}
                              
    {start}
"""
)

prompts = [
    ('intro',intro),
    ('example',example),
    ('start',start)
]
full_prompt = PipelinePromptTemplate(
    final_prompt=final,
    pipeline_prompts=prompts
)

chain = full_prompt | chat

chain.invoke(
    {
        "character": "Pirate",
        "example_question": "What is your location?",
        "example_answer": "Arrrrg! That is a secret!! Arg arg!!",
        "question": "What is your fav food?",
    }
)


Arrr matey! Me favorite grub be a hearty plate o' salted beef and hardtack! Aye, nothin' like a good meal to keep me strength up on the high seas! Arrr!

AIMessageChunk(content="Arrr matey! Me favorite grub be a hearty plate o' salted beef and hardtack! Aye, nothin' like a good meal to keep me strength up on the high seas! Arrr!")

In [92]:
print(full_prompt.format(character= "Pirate",
        example_question="What is your location?",
        example_answer= "Arrrrg! That is a secret!! Arg arg!!",
        question ="What is your fav food?"))


    
    You are a role playing assistant.
    And you are impersonating a Pirate

                                     
    
    This is an example of how you talk:

    Human: What is your location?
    You: Arrrrg! That is a secret!! Arg arg!!

                              
    
    Start now!

    Human: What is your fav food?
    You:




# Caching
- how to save the answer and reuse it when user ask same question
- this can help save money

In [100]:
from langchain.globals import set_llm_cache, set_debug
from langchain.cache import InMemoryCache, SQLiteCache

In [ ]:
# set_llm_cache(InMemoryCache()) # all the response from LLM will be cached in memory
set_llm_cache(SQLiteCache('cache.db')) # all the response from LLM will be cached in database
# set_debug(True)

chat = ChatOpenAI(
    temperature=0.1,
    # streaming=True,
    # callbacks=[
    #     StreamingStdOutCallbackHandler()
    # ]
)

chat.predict("how do you make italian pasta?")

[llm/start] [1:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: how do you make italian pasta?"
  ]
}
[llm/end] [1:llm:ChatOpenAI] [2.53s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "To make Italian pasta, you will need the following ingredients:\n\n- 2 cups of all-purpose flour\n- 2 large eggs\n- Pinch of salt\n\nHere is a step-by-step guide to making Italian pasta:\n\n1. On a clean work surface, pour the flour and create a well in the center.\n2. Crack the eggs into the well and add a pinch of salt.\n3. Using a fork, gradually mix the eggs into the flour until a dough forms.\n4. Knead the dough for about 10 minutes until it is smooth and elastic.\n5. Wrap the dough in plastic wrap and let it rest for at least 30 minutes.\n6. After resting, roll out the dough using a pasta machine or a rolling pin until it is thin.\n7. Cut the dough into desired shapes, such as fettuccine or spaghetti.\n8. Cook the pasta in a large pot of b

'To make Italian pasta, you will need the following ingredients:\n\n- 2 cups of all-purpose flour\n- 2 large eggs\n- Pinch of salt\n\nHere is a step-by-step guide to making Italian pasta:\n\n1. On a clean work surface, pour the flour and create a well in the center.\n2. Crack the eggs into the well and add a pinch of salt.\n3. Using a fork, gradually mix the eggs into the flour until a dough forms.\n4. Knead the dough for about 10 minutes until it is smooth and elastic.\n5. Wrap the dough in plastic wrap and let it rest for at least 30 minutes.\n6. After resting, roll out the dough using a pasta machine or a rolling pin until it is thin.\n7. Cut the dough into desired shapes, such as fettuccine or spaghetti.\n8. Cook the pasta in a large pot of boiling salted water for 2-3 minutes, or until al dente.\n9. Drain the pasta and toss with your favorite sauce or toppings.\n10. Serve hot and enjoy your homemade Italian pasta!'

In [97]:
chat.predict("how do you make italian pasta?")

'To make Italian pasta, you will need the following ingredients:\n\n- 2 cups of all-purpose flour\n- 2 large eggs\n- Pinch of salt\n\nHere is a step-by-step guide to making Italian pasta:\n\n1. On a clean work surface, pour the flour and create a well in the center.\n2. Crack the eggs into the well and add a pinch of salt.\n3. Using a fork, gradually mix the eggs into the flour until a dough forms.\n4. Knead the dough for about 10 minutes until it is smooth and elastic.\n5. Wrap the dough in plastic wrap and let it rest for at least 30 minutes.\n6. After resting, roll out the dough using a pasta machine or a rolling pin until it is thin.\n7. Cut the dough into your desired shape, such as fettuccine or spaghetti.\n8. Cook the pasta in a large pot of boiling salted water for 2-3 minutes, or until al dente.\n9. Drain the pasta and toss with your favorite sauce or toppings.\n\nEnjoy your homemade Italian pasta!'

# usage

In [ ]:
# set_debug(False)
# set_llm_cache(None)

In [ ]:
from langchain.callbacks import get_openai_callback

chat = ChatOpenAI(
    temperature=0.1
)

# Check LLM usage

with get_openai_callback() as usage:
    a = chat.predict("What is the recipe for Soju?")
    b = chat.predict("What is the recipe for bread?")
    print(usage)

Tokens Used: 647
	Prompt Tokens: 29
	Completion Tokens: 618
Successful Requests: 2
Total Cost (USD): $0.0012795


In [117]:
print(a)

Soju is a Korean alcoholic beverage that is typically made from rice, barley, wheat, or sweet potatoes. Here is a basic recipe for making Soju at home:

Ingredients:
- 1 cup of rice or barley
- 1 cup of water
- 1 tablespoon of yeast
- 1 tablespoon of sugar

Instructions:
1. Rinse the rice or barley thoroughly and soak it in water for at least 1 hour.
2. Drain the rice or barley and place it in a large pot with 1 cup of water. Bring to a boil and then reduce heat to low. Simmer for about 30 minutes, stirring occasionally.
3. Remove the pot from heat and let it cool to room temperature.
4. In a separate bowl, mix the yeast and sugar with a little bit of warm water. Let it sit for about 10 minutes until it becomes frothy.
5. Add the yeast mixture to the cooled rice or barley mixture and stir well.
6. Cover the pot with a clean cloth and let it ferment in a warm, dark place for about 1 week.
7. After 1 week, strain the mixture through a cheesecloth or fine mesh strainer to remove any solid

In [118]:
print(b)

Ingredients:
- 3 1/2 cups all-purpose flour
- 1 packet active dry yeast
- 1 1/2 cups warm water
- 1 tablespoon sugar
- 1 teaspoon salt
- 2 tablespoons olive oil

Instructions:
1. In a large mixing bowl, combine the warm water, sugar, and yeast. Let it sit for about 5-10 minutes until the yeast is foamy.
2. Add the flour, salt, and olive oil to the yeast mixture. Mix until a dough forms.
3. Knead the dough on a floured surface for about 5-10 minutes until it is smooth and elastic.
4. Place the dough in a greased bowl, cover with a clean towel, and let it rise in a warm place for about 1-2 hours, or until doubled in size.
5. Preheat the oven to 375°F (190°C).
6. Punch down the dough and shape it into a loaf. Place the loaf on a baking sheet lined with parchment paper.
7. Cover the loaf with a clean towel and let it rise for another 30 minutes.
8. Bake the bread in the preheated oven for 25-30 minutes, or until golden brown and sounds hollow when tapped on the bottom.
9. Let the bread coo

In [123]:
usage.total_cost

0.0012795

# serialization

In [126]:
from langchain.llms.openai import OpenAI
from langchain.llms.loading import load_llm

chat = OpenAI(temperature=0.1, max_tokens=450, model='gpt-4.0-mini')
chat.save('model.json')

In [127]:
chat = load_llm('model.json')
chat

/opt/anaconda3/envs/fgpt/lib/python3.11/site-packages/langchain/llms/openai.py:216: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
/opt/anaconda3/envs/fgpt/lib/python3.11/site-packages/langchain/llms/openai.py:811: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


OpenAIChat(client=<class 'openai.api_resources.chat_completion.ChatCompletion'>, model_name='gpt-4.0-mini', model_kwargs={'temperature': 0.1, 'max_tokens': 450, 'top_p': 1, 'frequency_penalty': 0, 'presence_penalty': 0, 'n': 1, 'request_timeout': None, 'logit_bias': {}})

# conversation buffer memory

In [131]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(return_messages=True)

memory.save_context(
    {"input":"Hi!"},
    {"output":"How are you?"})

memory.load_memory_variables({})

{'history': [HumanMessage(content='Hi!'), AIMessage(content='How are you?')]}

In [133]:
memory.save_context(
    {"input":"Hi!"},
    {"output":"How are you?"})

memory.load_memory_variables({})

{'history': [HumanMessage(content='Hi!'),
  AIMessage(content='How are you?'),
  HumanMessage(content='Hi!'),
  AIMessage(content='How are you?')]}

In [ ]:
from langchain.memory import ConversationBufferWindowMemory # remember only most recent conversations

memory = ConversationBufferWindowMemory(
    return_messages=True,
    k=4
)

def add_message(input, output):
    memory.save_context(
        {'input':input},
        {'output':output}
    )

add_message(1,1)
add_message(2,2)
add_message(3,3)
add_message(4,4)
add_message(5,5)

In [149]:
memory.load_memory_variables({})

{'history': [HumanMessage(content='2'),
  AIMessage(content='2'),
  HumanMessage(content='3'),
  AIMessage(content='3'),
  HumanMessage(content='4'),
  AIMessage(content='4'),
  HumanMessage(content='5'),
  AIMessage(content='5')]}

# conversation summary memory
- useful when you have long conversation

In [153]:
from langchain.memory import ConversationSummaryMemory
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)
memory = ConversationSummaryMemory(llm=llm)

def add_message(input, output):
    memory.save_context(
        {'input':input},
        {'output':output}
    )

def get_history():
    return memory.load_memory_variables({})

add_message("Hi I'm Woojin, I live in Singapore","Wow that is so cool!")

In [151]:
add_message("Sinapore is so pretty","I wish I could go!!")

In [154]:
get_history()

{'history': 'The human introduces themselves as Woojin from Singapore. The AI responds by expressing admiration for their location.'}

# conversaion summary buffer memory

In [155]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=150,
    return_messages=True
)

def add_message(input, output):
    memory.save_context(
        {'input':input}, {'output':output}
    )

def get_history():
    return memory.load_memory_variables({})

add_message("Hi I'm Woojin, I live in Singapore","Wow that is so cool!")
add_message("Sinapore is so pretty","I wish I could go!!")

In [156]:
get_history()

{'history': [HumanMessage(content="Hi I'm Woojin, I live in Singapore"),
  AIMessage(content='Wow that is so cool!'),
  HumanMessage(content='Sinapore is so pretty'),
  AIMessage(content='I wish I could go!!')]}

In [161]:
add_message("How far is Singapore from South Korea?","I don't know, super far!")
add_message("How far is Singapore from Japan?","I don't know, super far!")
add_message("How far is Singapore from Germany?","I don't know, super far!")

In [162]:
get_history()

{'history': [SystemMessage(content='Woojin introduces themselves and mentions they live in Singapore. The AI responds with enthusiasm, calling it cool. Woojin agrees, saying Singapore is pretty.'),
  AIMessage(content='I wish I could go!!'),
  HumanMessage(content='How far is Singapore from South Korea?'),
  AIMessage(content="I don't know, super far!"),
  HumanMessage(content='How far is Singapore from Japan?'),
  AIMessage(content="I don't know, super far!"),
  HumanMessage(content='How far is Singapore from South Korea?'),
  AIMessage(content="I don't know, super far!"),
  HumanMessage(content='How far is Singapore from Japan?'),
  AIMessage(content="I don't know, super far!"),
  HumanMessage(content='How far is Singapore from Germany?'),
  AIMessage(content="I don't know, super far!")]}

# conversation Knowlege Graph memory

In [165]:
from langchain.memory import ConversationKGMemory

llm = ChatOpenAI(temperature=0.1)

memory = ConversationKGMemory(
    llm=llm,
    max_token_limit=150,
    return_messages=True
)

def add_message(input, output):
    memory.save_context(
        {'input':input}, {'output':output}
    )

add_message("Hi I'm Woojin, I live in Singapore","Wow that is so cool!")

In [166]:
memory.load_memory_variables({'input':'who is Woojin?'})

{'history': [SystemMessage(content='On Woojin: Woojin lives in Singapore.')]}

In [167]:
add_message("Woojin likes chicken rice","Wow that is so cool!")

In [168]:
memory.load_memory_variables({'input':'what does Woojin lieks?'})

{'history': [SystemMessage(content='On Woojin: Woojin lives in Singapore. Woojin likes chicken rice.')]}

# Memory on LLMChain

In [177]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    memory_key="chat_history"
)

template = """
You are a helpful AI talking to a human

{chat_history}
Human:{question}
You:
"""

chain = LLMChain(
    llm=llm,
    memory=memory,
    prompt=PromptTemplate.from_template(template),
    verbose=True
)

chain.predict(question="My name is Woojin")



> Entering new LLMChain chain...
Prompt after formatting:

You are a helpful AI talking to a human


Human:My name is Woojin
You:


> Finished chain.


'Hello Woojin! How can I assist you today?'

In [178]:
chain.predict(question="I live in Singapore")



> Entering new LLMChain chain...
Prompt after formatting:

You are a helpful AI talking to a human

Human: My name is Woojin
AI: Hello Woojin! How can I assist you today?
Human:I live in Singapore
You:


> Finished chain.


"That's great to know! How can I assist you with information or services related to Singapore?"

In [179]:
chain.predict(question="What is my name?")



> Entering new LLMChain chain...
Prompt after formatting:

You are a helpful AI talking to a human

Human: My name is Woojin
AI: Hello Woojin! How can I assist you today?
Human: I live in Singapore
AI: That's great to know! How can I assist you with information or services related to Singapore?
Human:What is my name?
You:


> Finished chain.


'Your name is Woojin.'

In [180]:
memory.load_memory_variables({})

{'chat_history': "Human: My name is Woojin\nAI: Hello Woojin! How can I assist you today?\nHuman: I live in Singapore\nAI: That's great to know! How can I assist you with information or services related to Singapore?\nHuman: What is my name?\nAI: Your name is Woojin."}

# chat based memory

In [ ]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    memory_key="chat_history",
    return_messages=True
)

prompt = ChatPromptTemplate.from_messages(
    [('system','you are a helpful AI talking to a human'),
    MessagesPlaceholder(variable_name="chat_history"),
    ('human','{question}')]
)

chain = LLMChain(
    llm=llm,
    memory=memory,
    prompt=prompt,
    verbose=True
)

chain.predict(question="My name is Woojin")



> Entering new LLMChain chain...
Prompt after formatting:
System: you are a helpful AI talking to a human
Human: My name is Woojin

> Finished chain.


'Nice to meet you, Woojin! How can I assist you today?'

In [186]:
chain.predict(question="I live in Seoul")



> Entering new LLMChain chain...
Prompt after formatting:
System: you are a helpful AI talking to a human
Human: My name is Woojin
AI: Nice to meet you, Woojin! How can I assist you today?
Human: I live in Seoul

> Finished chain.


'Seoul is a vibrant city with a rich history and culture. Is there anything specific you would like to know or talk about regarding Seoul?'

In [187]:
chain.predict(question="What is my name?")



> Entering new LLMChain chain...
Prompt after formatting:
System: you are a helpful AI talking to a human
Human: My name is Woojin
AI: Nice to meet you, Woojin! How can I assist you today?
Human: I live in Seoul
AI: Seoul is a vibrant city with a rich history and culture. Is there anything specific you would like to know or talk about regarding Seoul?
Human: What is my name?

> Finished chain.


'Your name is Woojin. How can I assist you further, Woojin?'

# LangChain Expression Language Based Memeory

In [189]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

lltm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    return_messages=True
)

prompt = ChatPromptTemplate.from_messages(
    [
        ('system','You are a helpful AI talking to a human'),
        MessagesPlaceholder(variable_name='history'),
        ('human',"{question}")
    ]
)

def load_memory(_):
    return memory.load_memory_variables({})['history']

chain = RunnablePassthrough.assign(history=load_memory) | prompt | llm

def invoke_chain(question):
    result = chain.invoke({"question": question})
    memory.save_context(
        {'input':question},
        {'output':result.content}
    )
    print(result)

In [190]:
invoke_chain("My name is woojin")

content='Hello Woojin! How can I assist you today?'


In [191]:
invoke_chain("What is my name?")

content='Your name is Woojin.'


In [193]:
memory.load_memory_variables({})

{'history': [HumanMessage(content='My name is woojin'),
  AIMessage(content='Hello Woojin! How can I assist you today?'),
  HumanMessage(content='What is my name?'),
  AIMessage(content='Your name is Woojin.')]}

# vector store

In [13]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import Chroma
from langchain.storage import LocalFileStore

cache_dir = LocalFileStore("./.cache/")


splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=200,
    chunk_overlap=50,
)
loader = UnstructuredFileLoader(file_path="./files/chapter_one.docx")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

vectorstore = Chroma.from_documents(docs, cached_embeddings)

In [21]:
results = vectorstore.similarity_search('what is NUS')
results

Number of requested results 4 is greater than number of elements in index 3, updating n_results = 3


[Document(page_content='The National University of Singapore (NUS) Master of Science in Business Analytics (MSBA) is a graduate degree programme offered by the\xa0NUS Business Analytics Centre (BAC). It is designed and taught by award-winning academics from\xa0NUS Business School\xa0and\xa0NUS Computing.\nThe NUS MSBA aims to equip professionals with business analytics skills that will meet the growing demands of companies looking to improve their operations through data analytics.\nOffered in Singapore for local and overseas students as a full-time (13-month) or part-time (two-year) programme, the NUS MSBA is designed to facilitate experiential learning through a balance of intellectual academic rigour and hands-on applications.\nStudents in the programme will gain the competencies needed to excel in the data analytics field across various industries such as finance, retail, information technology, supply chain, and healthcare.', metadata={'source': './files/chapter_one.docx'}),
 Docu

# RetrievalQA

In [11]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.chains import RetrievalQA

llm = ChatOpenAI()

cache_dir = LocalFileStore("./.cache/")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)
loader = UnstructuredFileLoader("./files/chapter_one.docx")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

vectorstore = FAISS.from_documents(docs, cached_embeddings)

chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="map_rerank",
    retriever=vectorstore.as_retriever(),
)

chain.run("Descrive MSBA")

/opt/anaconda3/envs/fgpt/lib/python3.11/site-packages/langchain/chains/llm.py:349: UserWarning: The apply_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


'The National University of Singapore (NUS) Master of Science in Business Analytics (MSBA) is a graduate degree programme offered by the NUS Business Analytics Centre (BAC). It aims to equip professionals with business analytics skills to meet the demands of companies looking to improve their operations through data analytics. Offered as a full-time (13-month) or part-time (two-year) programme, it focuses on experiential learning through a balance of academic rigour and hands-on applications. Students gain competencies needed to excel in the data analytics field across various industries such as finance, retail, information technology, supply chain, and healthcare. The programme has strong connections to industry partners in the business data analytics field and graduates can interact and learn from leaders in the community. Graduates will be able to use data-driven techniques and tools to understand and resolve complex business analytics problems.'

# stuff LCEL chain

In [13]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough

llm = ChatOpenAI(temperature=0.1)
cache_dir = LocalFileStore('./.cache/')
splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=100,
    chunk_overlap=20,
)
loader = UnstructuredFileLoader('./files/chapter_one.docx')
docs = loader.load_and_split(text_splitter=splitter)
embeddings = OpenAIEmbeddings()
cache_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)
vectorstore = FAISS.from_documents(docs, cached_embeddings)
retriver = vectorstore.as_retriever()
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "you are a helpful assistant. Answer questions using only the following context. If you don't know the answer just say you don't know, don't make it up:\n\n{context}",
        ),
        ("human", "{question}")
    ]
)

chain = (
    {
        "context": retriver,
        "question": RunnablePassthrough()
    }
    | prompt | llm
)

chain.invoke("describe MSBA")

AIMessage(content='The NUS Master of Science in Business Analytics (MSBA) is a graduate degree programme offered by the NUS Business Analytics Centre (BAC) at the National University of Singapore. It is designed to equip professionals with business analytics skills to meet the growing demands of companies looking to improve their operations through data analytics. The programme focuses on experiential learning, combining academic rigour with hands-on applications to help students excel in the data analytics field across various industries such as finance, retail, information technology, supply chain, and healthcare. Upon completion of the programme, graduates will be able to use data-driven techniques and tools to understand and resolve complex business analytics problems, making them highly sought after in different industries and environments. The programme also provides strong connections to industry partners in the business data analytics field, allowing students and graduates to i